In [1]:
from typing import Any, Protocol, runtime_checkable, TypeVar, Set, Tuple
from queue import Queue
from transitions import Machine
from copy import deepcopy
import tcod

#################################################################
#  Atlas Display Roster                                         #
#    |     |      |                Inputs                       #
#   Map Widget Entity                |                          #
#    \    |    /                     |                          #
#    Game States ___ SMs(AI) __ +Game Event(AI) __ Handle(AI)   #
#         |                          |               |          #
#   Action Perform ______________ Reactions   State Event Queue #      
#         |                                          |          #                 
#   "Dispatch" Thread                         "Handle" Thread   #
#         |                                          |          #
#    State Action Queue                         Event Trigger   #
#         |                                          |          #
#         |___________________+Game Action(AI) __ Dispatch(AI)  #
#                                                               #
# ############################################################### 

@runtime_checkable
class StatefulObject(Protocol):
    """The StatefulObject Protocol is a mixin class that has a 'machine' attribute."""
    machine: Machine# Has a state machine that defines states and transitions.

@runtime_checkable
class StateReferenceObject(Protocol):
    """The StateReferenceObject Protocol is a mixin class that has a 'state' attribute. It is used to refer to a StatefulObject instance for context."""
    state: StatefulObject | None 

@runtime_checkable
class StateActionObject(Protocol):
    """
    The StateActionObject Protocol is a mixin class that has a 'state' and 'transformer' attributes.
    
    Duck Types: StatefulObject
    
    """
    state: StatefulObject | None 
    transformer: Any | None

@runtime_checkable
class StateBehaviorObject(Protocol):
    """The StatefulBehaviorObject Protocol is a mixin class that has a 'state' and 'behaviors' attribute."""
    behaviors: Set[Tuple[str, StateActionObject]] | None # Has a set of behaviors (events/actions) that can be queued for execution by the game engine.

@runtime_checkable
class StateTransformer(StateReferenceObject, StateBehaviorObject,Protocol):
    """
    The StateTransformer Protocol contains StateActionObject mappings (behaviors) and methods for queuing them
     in the Queue component of the STO State component. The execution flow is, given an STO as an input, the StateTransfor 
     uses it to 'get_behavior' from the mapping, uses the STO's State component to 'set_context' on the behavior, then 
     'send_behavior' enqueues the contextualized behavior into the State component's Queue.
     
     Duck Types: StateReferenceObject, StateBehaviorObject
     """

    T = TypeVar('T', bound=StateActionObject)
    
    def _transform(self, name: str) -> StateActionObject | None:
        """Retrieves a behavior (GameEvent or GameAction) by name from the behaviors set."""
        ...
    
    def _set_context(self, action: T, *args, **kwargs) -> T:
        """Must be overidden by subclasses. Sets the context of the StateActionObject item with the arguments."""
        ...

    def _send(self, action: StateActionObject)  -> bool:
        "This method should be overidden by subclasses. Enqueues the contextualized input_item into a GameState queue."
        ...
    


In [2]:
class BaseGameState:
    actions: Queue[StateActionObject] | None
    events: Queue[StateActionObject] | None
    roster: StatefulObject | None
    atlas: StatefulObject | None
    display: StatefulObject | None
    machine: Machine


class BaseLoopHandler:
    """The GameHandler is responsible for tranforming Game Inputs and AI Actions into Game Events
    and sending them to the State Action Queue."""
    state: StatefulObject | None 
    behaviors: Set[Tuple[str, StateActionObject]] | None # Has a set of behaviors (events/actions) that can be queued for execution by the game engine.

    def __init__(self, state: StatefulObject | None = None, behaviors: Set[Tuple[str, StateActionObject]] | None = None) -> None:
        self.state = state
        self.behaviors = behaviors

    T = TypeVar('T', bound=StateActionObject)
    
    def _transform(self, name: str) -> StateActionObject | None:
        """Should NOT be overidden by subclasses. Retrieves a behavior (Event or Action) by name from the behaviors set."""
        if self.behaviors is not None:
            for behavior in self.behaviors:
                if name == behavior[0]:
                    action = behavior[1]
                    return deepcopy(action)

            raise ValueError(f"Behavior not found for event: {name}")
        raise ValueError(f"State behaviors object not found: {name}")
    
    def _set_context(self, action: T, *args, **kwargs) -> T:
        """Can be overidden by subclasses. Sets the context of the StateActionObject item with the arguments."""
        action.state = self.state
        action.transformer = self
        return action

    def _send(self, action: StateActionObject)  -> bool:
        "This method must be overidden by subclasses. Enqueues the contextualized input_item into a GameState queue."
        ...

    def _transform_send(self, event: StateActionObject | tcod.event.Event) -> bool:
        try:
            action = self._transform(event.__class__.__name__.lower())
            contextualized_action = None
            if action:
                contextualized_action = self._set_context(action)
            
            if contextualized_action:
                return self._send(contextualized_action)
            else:
                return False
            
        except Exception as e:
            print(f"Dispatch error: {e}")
            return False

    # def handle(self, event: StateActionObject | tcod.event.Event | None = None) -> bool:
    #     if event is not None:
    #             return self._transform_send(event)
        
    #     return False
    

class BaseGameAction:
    state: StatefulObject | None
    transformer: BaseLoopHandler | None
    
    def __init__(self, state: BaseGameState | None = None, transformer: BaseLoopHandler | None = None) -> None:
        self.state = state
        self.transformer = transformer

    def perform(self) -> None:
        raise NotImplementedError("Subclasses must implement the perform method.")


class BaseGameEvent:
    state: StatefulObject | None
    transformer: BaseLoopHandler | None
    
    def __init__(self, state: BaseGameState | None = None, transformer: BaseLoopHandler | None = None) -> None:
        self.state = state
        self.transformer = transformer

    def trigger(self) -> None:
        if self.transformer:
            self.transformer._transform_send(self)
        


In [ ]:
class GameState(BaseGameState):
    """ The Game State maintains the current state of the game, including the action and event queues,
    as well as references to the display roster and atlas. 
    
    Duck Types: BaseGameState, StatefulObject
    """
    
    def __init__(self):
        super().__init__()
        self.actions = Queue()
        self.events = Queue()
        self.roster = None
        self.atlas = None
        self.display = None
        
        states = ['idle', 
                  {'name': 'started', 'on_enter': 'game_init'}, 
                  {'name': 'playing', 'on_enter': 'game_start'}, 
                  {'name': 'stopped', 'on_enter': 'game_stop'}]
        
        transitions =[
            {'trigger': 'start_game', 'source': 'idle', 'dest': 'started'},
            {'trigger': 'play_game', 'source': 'started', 'dest': 'playing'},
            {'trigger': 'stop_game', 'source': 'started', 'dest': 'stopped'},
            {'trigger': 'stop_game', 'source': 'playing', 'dest': 'stopped'},
            {'trigger': 'reset_game', 'source': 'stopped', 'dest': 'started'},
            ]
        
        self.machine = Machine(model=self, states=states, transitions=transitions, initial='idle')

    def game_init(self):
        """Initializes the game state when the game starts."""
        print("Game is initializing...")
    
    def game_start(self):
        """Starts the game loop and prepares the game state for play."""
        print("Game has started. Let the adventure begin!")

    def game_stop(self):
        """Handles any cleanup or finalization needed when the game stops."""
        print("Game has been stopped. Performing cleanup...")

a = GameState()
print(a.start_game()) # type: ignore
print(isinstance(a, BaseGameState), isinstance(a, StatefulObject), isinstance(a, StateReferenceObject), isinstance(a, StateActionObject), isinstance(a, StateBehaviorObject), isinstance(a, StateTransformer))  
print(a.stop_game()) # type: ignore


Game is initializing...
True
True True True False False False
Game has been stopped. Performing cleanup...
True


In [4]:
a = BaseLoopHandler()
isinstance(a, StatefulObject), isinstance(a, StateReferenceObject), isinstance(a, StateActionObject), isinstance(a, StateBehaviorObject), isinstance(a, StateTransformer)


(False, True, False, True, True)

In [5]:
### EVENTS ###

class NonEvent(BaseGameEvent):
    
    def trigger(self) -> None:
        print("Nothing happened.")
        super().trigger()

class GameStopEvent(BaseGameEvent):

    def trigger(self) -> None:
        print("Game stopping...")
        super().trigger()

class GameStartEvent(BaseGameEvent):

    def trigger(self) -> None:
        print("Game starting...")
        super().trigger()       

class GamePlayEvent(BaseGameEvent):

    def trigger(self) -> None:
        print("Game playing...")
        super().trigger()


a = NonEvent()
isinstance(a, BaseGameEvent), isinstance(a, BaseGameAction), isinstance(a, StatefulObject), isinstance(a, StateReferenceObject), isinstance(a, StateActionObject), isinstance(a, StateBehaviorObject), isinstance(a, StateTransformer)   

(True, False, False, True, True, False, False)

In [11]:
### ACTIONS ###

class NoAction(BaseGameAction):

    def perform(self) -> None:
        event = self.transformer._transform('nonevent') # type: ignore
        self.transformer.handle(event) # type: ignore

        print("No action performed, no reaction events")


class GameStopAction(BaseGameAction):

    def perform(self) -> None:
       if self.state is not None:
           self.state.stop_game() # type: ignore

class GameStartAction(BaseGameAction):

    def perform(self) -> None:
       if self.state is not None:
           self.state.start_game() # type: ignore

class GamePlayAction(BaseGameAction):
    
    def perform(self) -> None:
       if self.state is not None:
           self.state.play_game() # type: ignore

a = NoAction()
isinstance(a, BaseGameEvent), isinstance(a, BaseGameAction),isinstance(a, StatefulObject), isinstance(a, StateReferenceObject), isinstance(a, StateActionObject), isinstance(a, StateBehaviorObject), isinstance(a, StateTransformer)   


(False, True, False, True, True, False, False)

In [12]:
import tcod

class GameLoopHandler(BaseLoopHandler):
    """The GameLoopHandler is responsible for tranforming Game Inputs and AI Actions into Game Events
    and sending them to the State Action Queue."""
    T = TypeVar('T', bound=StateActionObject)

    def __init__(self):
        game_state = GameState()
        game_behaviors = {
                        ('noaction', NonEvent()),
                        ('nonevent', NoAction()),
                        ('gamestopevent', GameStopAction()),
                        ('gamestartevent', GameStartAction()),
                        ('gameplayevent', GamePlayAction()),
                        }
        super().__init__(state=game_state, behaviors=game_behaviors)
    
    def _send(self, action: StateActionObject)  -> bool:
        try:
            active_queue = None

            if isinstance(action, BaseGameEvent) and isinstance(action.state, BaseGameState):
                active_queue = action.state.events
            elif isinstance(action, BaseGameAction) and isinstance(action.state, BaseGameState):
                active_queue = action.state.actions

            if active_queue is not None:
                active_queue.put(action)
                return True
                    
            return False
        except Exception as e:
            raise e
        
    def handle(self, event: BaseGameEvent | tcod.event.Event | None = None) -> bool:
        if event is not None:
                return self._transform_send(event)
        
        return False

In [ ]:
game_state = GameState()
game_state.actions = Queue()
game_state.events = Queue()

handler = GameLoopHandler()
handler.state = game_state

event = NonEvent(transformer=handler)
event.transformer = handler

handler._transform_send(event)
i = 1

while i < 10:
    if game_state.actions:
        new_action = game_state.actions.get_nowait()
        if isinstance(new_action, BaseGameAction):
            new_action.perform()

        new_event = game_state.events.get_nowait()
        if isinstance(new_event, BaseGameEvent):
            new_event.trigger()
        i += 1

In [13]:
game_state = GameState()
game_state.actions = Queue()
game_state.events = Queue()

handler = GameLoopHandler()
handler.state = game_state

event = GameStartEvent(state=game_state, transformer=handler)
event.trigger()

game_state.events.qsize(),  game_state.actions.qsize()


Game starting...


(0, 1)

In [14]:
new_action = game_state.actions.get_nowait()
if isinstance(new_action, BaseGameAction):
    new_action.perform()
    
game_state.state

Game is initializing...


'started'